In [1]:
import sys, os, distutils.core

In [2]:
run install_detectron2.ipynb

Obtaining file:///aiffel/aiffel/a_project/viton/detectron2
  Preparing metadata (setup.py) ... done


  Attempting uninstall: detectron2
    Found existing installation: detectron2 0.6
    Uninstalling detectron2-0.6:
      Successfully uninstalled detectron2-0.6
  Running setup.py develop for detectron2


In [3]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import os
import torch as nn

from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg

from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode

In [4]:
nn.cuda.empty_cache()

In [5]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog

register_coco_instances('fashion_train', {}, 'Fashionpedia/train_annotation.json', 'Fashionpedia/train_img/')
register_coco_instances('fashion_val', {}, 'Fashionpedia/test_annotation.json', 'Fashionpedia/test_img/')




#from detectron2.data import get_detection_dataset_dicts
#from detectron2.data.datasets import builtin_meta



In [6]:
fashion_metadata = MetadataCatalog.get("fashion_train")
dataset_dicts = DatasetCatalog.get('fashion_train')

[03/30 14:21:29 d2.data.datasets.coco]: Loading Fashionpedia/train_annotation.json takes 5.21 seconds.
[03/30 14:21:29 d2.data.datasets.coco]: Loaded 45602 images in COCO format from Fashionpedia/train_annotation.json


In [7]:
#dataset_dicts[3]

In [8]:
#plt.imshow(a)

In [9]:
from detectron2 import model_zoo

cfg = get_cfg()

In [10]:
cfg.merge_from_file("detectron2/configs/Misc/scratch_mask_rcnn_R_50_FPN_3x_gn.yaml")

In [11]:

cfg.DATASETS.TRAIN = ("fashion_train",)
cfg.DATASETS.TEST = ('fashion_val',)  
cfg.DATALOADER.NUM_WORKERS = 1
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth") 
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.12
cfg.SOLVER.MAX_ITER = (
    1000
)  # 300 iterations seems good enough, but you can certainly train longer
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (
    4
)  # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 47    #47
cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 47    #47
cfg.INPUT.MASK_FORMAT='bitmask'
cfg.TEST.EVAL_PERIOD = 0 
cfg.SOLVER.CHECKPOINT_PERIOD = 100
cfg.INPUT.MIN_SIZE_TRAIN = (500,)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [12]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[03/30 14:21:39 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(
      256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
      (norm): GroupNorm(32, 256, eps=1e-05, affine=True)
    )
    (fpn_output2): Conv2d(
      256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (norm): GroupNorm(32, 256, eps=1e-05, affine=True)
    )
    (fpn_lateral3): Conv2d(
      512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
      (norm): GroupNorm(32, 256, eps=1e-05, affine=True)
    )
    (fpn_output3): Conv2d(
      256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (norm): GroupNorm(32, 256, eps=1e-05, affine=True)
    )
    (fpn_lateral4): Conv2d(
      1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
      (norm): GroupNorm(32, 256, eps=1e-05, affine=True)
    )
    (fpn_output4): Conv2d(
      256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (norm): Group

[03/30 14:21:43 d2.data.datasets.coco]: Loading Fashionpedia/train_annotation.json takes 4.46 seconds.
[03/30 14:21:43 d2.data.datasets.coco]: Loaded 45602 images in COCO format from Fashionpedia/train_annotation.json
[03/30 14:21:45 d2.data.build]: Removed 0 images with no usable annotations. 45602 images left.
[03/30 14:21:46 d2.data.build]: Distribution of instances among all 22 categories:
|   category    | #instances   |   category    | #instances   |   category    | #instances   |
|:-------------:|:-------------|:-------------:|:-------------|:-------------:|:-------------|
| shirt, blouse | 6161         | top, t-shir.. | 16548        |    sweater    | 1494         |
|   cardigan    | 1107         |    jacket     | 7833         |     vest      | 719          |
|     pants     | 12414        |    shorts     | 2756         |     skirt     | 5046         |
|     coat      | 3124         |     dress     | 18739        |   jumpsuit    | 922          |
|    glasses    | 4855         | 

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (24, 1024) in the checkpoint but (48, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (24,) in the checkpoint but (48,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (92, 1024) in the checkpoint but (188, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (92,) in the checkpoint but (188,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (23, 256, 1, 1) in the checkpoint but (47, 256, 1, 

[03/30 14:22:12 d2.engine.train_loop]: Starting training from iteration 0
[03/30 14:22:29 d2.utils.events]:  eta: 0:11:57  iter: 19  total_loss: 5.323  loss_cls: 3.681  loss_box_reg: 1.018  loss_mask: 0.6835  loss_rpn_cls: 0.02455  loss_rpn_loc: 0.01972    time: 0.7501  last_time: 0.5405  data_time: 0.1851  last_data_time: 0.0075   lr: 0.0023977  max_mem: 7210M
[03/30 14:22:44 d2.utils.events]:  eta: 0:11:33  iter: 39  total_loss: 3.064  loss_cls: 1.35  loss_box_reg: 1.01  loss_mask: 0.5666  loss_rpn_cls: 0.02767  loss_rpn_loc: 0.02816    time: 0.7489  last_time: 0.9777  data_time: 0.1567  last_data_time: 0.1760   lr: 0.0047953  max_mem: 7210M
[03/30 14:22:59 d2.utils.events]:  eta: 0:11:21  iter: 59  total_loss: 2.631  loss_cls: 1.178  loss_box_reg: 0.9337  loss_mask: 0.4272  loss_rpn_cls: 0.0311  loss_rpn_loc: 0.02309    time: 0.7476  last_time: 0.7699  data_time: 0.1608  last_data_time: 0.2055   lr: 0.0071929  max_mem: 7210M
[03/30 14:23:15 d2.utils.events]:  eta: 0:11:19  iter: 79 

KeyboardInterrupt: 